# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio
import io
import os

from PIL import Image
import requests
import sglang as sgl

from sglang.srt.conversation import chat_templates
from sglang.test.test_utils import is_in_ci
from sglang.utils import async_stream_and_merge, stream_and_merge

if is_in_ci():
    import patch
else:
    import nest_asyncio

    nest_asyncio.apply()


llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.73it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.72it/s]



### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Kiera. I'm a young, ambitious marketing executive with a passion for helping businesses grow and thrive. My goal is to help people discover and implement creative solutions to their marketing and communication problems.
Before I became a marketing executive, I studied Business Administration at the University of Waterloo, where I specialized in Marketing. I have a deep understanding of the marketing industry, including how to create compelling content, design effective campaigns, and deliver results.
I have over 7 years of experience in the marketing industry, and my expertise extends to digital marketing, brand management, content creation, and social media. I am also a certified public speaker and a member
Prompt: The president of the United States is
Generated text:  a powerful role holder, but he is also the president of the executive branch of the government. A presidential administration is a series of presidential administrations, usual

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a brief description of your profession or role]. I enjoy [insert a brief description of your hobbies or interests]. What brings you to [company name] and what makes you a good fit for the position? I'm a [insert a brief description of your personality or traits]. I'm always looking for new challenges and opportunities to grow and learn. What do you think makes [company name] a great place to work? I'm excited

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

The statement is concise and accurately describes the capital city of France, providing only the key information needed to understand its location and significance. It does not include any additional details or context that would not be necessary to fully understand the statement. 

The statement is also clear and easy to read, making it suitable for use in various contexts, such as news articles, travel guides, or educational materials. It is also concise enough to be easily remembered by readers who have not encountered the capital city before. 

Overall, the statement effectively communicates the key information about Paris's capital status and importance to the French people and the world. It is

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes.

2. Enhanced emotional intelligence: AI systems are likely to become more capable of understanding and responding to human emotions, leading to more empathetic and compassionate AI.

3. Increased use of AI in healthcare: AI is likely to play a more significant role in healthcare, with more personalized and accurate diagnoses and treatments.

4. Greater use of AI in transportation: AI is likely to become more integrated with transportation, with more efficient and reliable



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Character Name], I'm a/an [Number] year [Number] year old. I'm from [City or Country]. I love [What I'm passionate about or what I enjoy doing]. I have a/an [Number] degree in [Field or Subject] and I've been [Number] years a professional in [Role], but I haven't worked in [Role] for a while yet. I enjoy [What I enjoy doing], and I'm always learning. I'm [Number] years old now, but I hope to get older, and I hope to achieve something meaningful in my career. How about you? What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the world’s most populous city, located in the heart of the city of Paris.

That's a good start! Here's a concise and factual statement:

Paris, the world's most populous city, serves as the capital of France. 

Is there anything else you'd like me to include or expand on? 

If yo

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

,

 leading

 to

 new

 possibilities

In [6]:
llm.shutdown()